In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten,Conv2D,BatchNormalization,Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
key_points_df= pd.read_csv('data/training_frames_keypoints.csv')

def make_dataset(csv_dir, root_dir):
    
    key_points_frame = pd.read_csv(csv_dir)
    x=[]
    y=[]
    
    for i in range(len(key_points_frame)):
        image_name= os.path.join(root_dir,key_points_frame.iloc[i,0])
        image = cv2.imread(image_name)
        image= image[:,:,0:3]
        #image= cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        h, w = image.shape[:2]
        image_resized = cv2.resize(image, (96,96),1)
        #image_resized = image_resized/255.0
        x.append(image_resized)
        
        key_points= key_points_frame.iloc[i, 1:].values
        key_points= key_points.astype('float').reshape(-1,2)
        key_points = key_points * [96/w, 96/h]
        key_points= key_points.reshape(-1)
        y.append(key_points)
        
    x = np.asarray(x, dtype=np.float32)
    y= np.asarray(y, dtype=np.float32)

    #sample={'image':x, 'points':y}
    return x, y

x_train, y_train= make_dataset("data/training_frames_keypoints.csv", "data/training/")

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [ ]:

print(X_train.shape)
print(X_test.shape)


In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters =32, kernel_size =(3,3), padding='same', input_shape=(96,96,3)))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters =32, kernel_size =(3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size =2))

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Conv2D(filters =64, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters =64, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters =128, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters =128, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(tf.keras.layers.Conv2D(filters =256, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters =256, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters =512, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters =512, kernel_size= (3,3),padding='same'))
model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=136))
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy', 'mae'])


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_weights.h5', monitor=['val_accuracy'],save_weights_only=True, mode='max', verbose=1)
reduce_lr= tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.9, patience=5, min_delta=0.00001, mode='auto')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
callbacks = [tensorboard_callback,checkpoint]

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 250 ,batch_size = 128, callbacks=callbacks)  


In [ ]:
model_json = model.to_json()
with open("model_new_latest.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
model.save_weights('model_weights_LATEST.h5')
print('model weights saved to disk')

In [ ]:
from tensorflow.keras.preprocessing import image
test_image = image.load_img("data/test/Abdullah_Gul_31.jpg", target_size=(96,96))
test_image1 = np.asarray(test_image, dtype=np.float32)
test_image1 = np.expand_dims(test_image, axis = 0)
print(test_image1.shape)



In [ ]:
result = model.predict(test_image1)
result= result.astype('float').reshape(-1,2)
plt.imshow(test_image)
plt.scatter(result[:, 0], result[:, 1], s=20, marker='.', c='m')

print(result)
